# Week 9-12 - Reflection

# Week 9 Reflection

**1. Write some code that will use a simulation to estimate the standard deviation of the coefficient when there is heteroskedasticity. Compare these standard errors to those found via statsmodels OLS or a similar linear regression model.**

In [7]:
import numpy as np

# Try to use statsmodels if present; otherwise fall back to manual formulas
try:
    import statsmodels.api as sm
    HAVE_SM = True
except Exception:
    HAVE_SM = False

rng = np.random.default_rng(42)

def one_draw(n=500, beta1=1.0, hetero_strength=2.0):
    """
    Generate one dataset with x ~ N(0,1), y = 1 + beta1*x + u,
    Var(u | x) = 1 + hetero_strength * x^2  (i.e., heteroskedastic).
    Returns slope estimate and SEs (conventional & robust).
    """
    x = rng.normal(size=n)
    var_u = 1.0 + hetero_strength * (x**2)     # heteroskedastic variance
    u = rng.normal(scale=np.sqrt(var_u))
    y = 1.0 + beta1 * x + u

    X = np.column_stack([np.ones(n), x])

    if HAVE_SM:
        model = sm.OLS(y, X).fit()
        # Conventional (homoskedastic) SE and robust (HC1) SE for slope
        se_homosked = model.bse[1]
        se_robust = model.get_robustcov_results(cov_type="HC1").bse[1]
        beta1_hat = model.params[1]
        return beta1_hat, se_homosked, se_robust

    # ------- Manual OLS + White(HC1) robust SEs -------
    XtX = X.T @ X
    XtX_inv = np.linalg.inv(XtX)
    beta_hat = XtX_inv @ (X.T @ y)
    resid = y - X @ beta_hat
    k = X.shape[1]

    # Conventional (homoskedastic) variance estimator:
    sigma2 = (resid @ resid) / (n - k)
    cov_homo = sigma2 * XtX_inv
    se_homosked = np.sqrt(cov_homo[1, 1])

    # White (HC0): (X'X)^(-1) X' diag(e^2) X (X'X)^(-1)
    Xe = X * resid[:, None]                 # avoids building an n×n diag
    S = Xe.T @ Xe
    cov_hc0 = XtX_inv @ S @ XtX_inv
    # HC1 finite-sample correction:
    cov_hc1 = cov_hc0 * (n / (n - k))
    se_robust = np.sqrt(cov_hc1[1, 1])

    beta1_hat = beta_hat[1]
    return beta1_hat, se_homosked, se_robust

def run_sim(R=1000, n=500, beta1=1.0, hetero_strength=2.0, verbose=True):
    """
    Repeat the simulation R times and compare:
      - empirical SD of beta1_hat
      - mean conventional SE
      - mean robust (HC1) SE
    """
    bhats = np.empty(R)
    se_h = np.empty(R)
    se_r = np.empty(R)
    for r in range(R):
        b, sh, sr = one_draw(n=n, beta1=beta1, hetero_strength=hetero_strength)
        bhats[r] = b
        se_h[r] = sh
        se_r[r] = sr

    empirical_sd = bhats.std(ddof=1)
    mean_homo = se_h.mean()
    mean_robust = se_r.mean()

    if verbose:
        print("=== Setup ===")
        print(f"R = {R}, n = {n}, true beta1 = {beta1}, hetero_strength = {hetero_strength}")
        print("\n=== Results (slope coefficient) ===")
        print(f"Empirical SD of beta1_hat       : {empirical_sd:8.4f}")
        print(f"Mean conventional OLS SE        : {mean_homo:8.4f}")
        print(f"Mean robust (White HC1) OLS SE  : {mean_robust:8.4f}")
        print("\nInterpretation:")
        print("- With heteroskedasticity, the conventional SE is typically biased (often too small).")
        print("- The robust (HC1) SE should be close to the empirical SD across simulations.")

    return {
        "empirical_sd": empirical_sd,
        "mean_homoskedastic_se": mean_homo,
        "mean_robust_se": mean_robust,
        "beta1_hats": bhats,
        "homosked_se_series": se_h,
        "robust_se_series": se_r
    }

# --- Run it ---
if __name__ == "__main__":
    _ = run_sim(R=1000, n=500, beta1=1.0, hetero_strength=2.0)

=== Setup ===
R = 1000, n = 500, true beta1 = 1.0, hetero_strength = 2.0

=== Results (slope coefficient) ===
Empirical SD of beta1_hat       :   0.1201
Mean conventional OLS SE        :   0.0774
Mean robust (White HC1) OLS SE  :   0.1174

Interpretation:
- With heteroskedasticity, the conventional SE is typically biased (often too small).
- The robust (HC1) SE should be close to the empirical SD across simulations.


**How to use/modify**

Heteroskedasticity strength: Increase hetero_strength to make variance depend more on x.

Sample size / repetitions: Change n and R to explore different regimes.

What to expect: In the presence of heteroskedasticity, the empirical SD of the slope estimates should align with the robust (HC1) mean SE, while the conventional SE is usually too optimistic (too small).

**The next codes show extended, self-contained script that adds coverage and size (Type-I error) checks for 95% CIs using both conventional and robust (HC1) standard errors under heteroskedasticity.**

- It still works with or without statsmodels (falls back to manual OLS + White HC1).

**- Reports:**

- **Empirical SD of** $\hat{\beta}_1$  
- **Mean conventional OLS SE vs robust (HC1) SE**  
- **95% confidence interval (CI) coverage** for both SE types  
- **Rejection rate (size) of the null hypothesis:**

$$
H_0:\ \beta_1 = \beta_{1,\text{true}} \quad \text{at } \alpha = 0.05
$$

---

**Notes:**

- Confidence intervals use the **normal critical value 1.96** by default.  
- This is appropriate when $n \ge 50$, because the $t$-distribution closely approximates the normal distribution.  
- You can switch to the **$t$-distribution** (instead of 1.96) if `scipy`

In [8]:
import numpy as np

# Try to use statsmodels if present; otherwise fall back to manual formulas
try:
    import statsmodels.api as sm
    HAVE_SM = True
except Exception:
    HAVE_SM = False

# Optional: if you want t-critical instead of 1.96, set USE_T=True and require scipy
USE_T = False
if USE_T:
    try:
        from scipy.stats import t
    except Exception:
        USE_T = False

rng = np.random.default_rng(42)

def one_draw(n=500, beta1=1.0, hetero_strength=2.0):
    """
    Generate one dataset with x ~ N(0,1), y = 1 + beta1*x + u,
    Var(u | x) = 1 + hetero_strength * x^2  (heteroskedastic).
    Returns beta1_hat and (conventional, robust) SEs.
    """
    x = rng.normal(size=n)
    var_u = 1.0 + hetero_strength * (x**2)     # heteroskedastic variance
    u = rng.normal(scale=np.sqrt(var_u))
    y = 1.0 + beta1 * x + u

    X = np.column_stack([np.ones(n), x])

    if HAVE_SM:
        model = sm.OLS(y, X).fit()
        se_homo = model.bse[1]
        se_rob = model.get_robustcov_results(cov_type="HC1").bse[1]
        return model.params[1], se_homo, se_rob

    # ------- Manual OLS + White(HC1) robust SEs -------
    XtX = X.T @ X
    XtX_inv = np.linalg.inv(XtX)
    beta_hat = XtX_inv @ (X.T @ y)
    resid = y - X @ beta_hat
    k = X.shape[1]

    # Conventional (homoskedastic) variance estimator:
    sigma2 = (resid @ resid) / (n - k)
    cov_homo = sigma2 * XtX_inv
    se_homo = np.sqrt(cov_homo[1, 1])

    # White (HC0): (X'X)^(-1) X' diag(e^2) X (X'X)^(-1)
    Xe = X * resid[:, None]
    S = Xe.T @ Xe
    cov_hc0 = XtX_inv @ S @ XtX_inv
    # HC1 correction
    cov_hc1 = cov_hc0 * (n / (n - k))
    se_rob = np.sqrt(cov_hc1[1, 1])

    return beta_hat[1], se_homo, se_rob

def run_sim(R=2000, n=500, beta1=1.0, hetero_strength=2.0, alpha=0.05, verbose=True):
    """
    Repeat the simulation R times and compare:
      - empirical SD of beta1_hat
      - mean conventional SE and mean robust (HC1) SE
      - 95% CI coverage for both SE types
      - size (Type I error) at alpha for two-sided z-test using both SE types
    """
    bhats = np.empty(R)
    se_h = np.empty(R)
    se_r = np.empty(R)

    for r in range(R):
        b, sh, sr = one_draw(n=n, beta1=beta1, hetero_strength=hetero_strength)
        bhats[r] = b
        se_h[r] = sh
        se_r[r] = sr

    empirical_sd = bhats.std(ddof=1)
    mean_homo = se_h.mean()
    mean_robust = se_r.mean()

    k = 2  # intercept + slope
    if USE_T:
        dof = n - k
        crit = t.ppf(1 - alpha/2, df=dof)
    else:
        crit = 1.959963984540054  # ~1.96

    # CI coverage for true beta1 using both SEs
    cover_h = ((bhats - crit*se_h) <= beta1) & (beta1 <= (bhats + crit*se_h))
    cover_r = ((bhats - crit*se_r) <= beta1) & (beta1 <= (bhats + crit*se_r))
    coverage_h = cover_h.mean()
    coverage_r = cover_r.mean()

    # Two-sided test of H0: beta1 = true value, reject if |t| > crit
    t_h = (bhats - beta1) / se_h
    t_r = (bhats - beta1) / se_r
    reject_h = (np.abs(t_h) > crit).mean()
    reject_r = (np.abs(t_r) > crit).mean()

    if verbose:
        print("=== Setup ===")
        print(f"R = {R}, n = {n}, true beta1 = {beta1}, hetero_strength = {hetero_strength}, alpha = {alpha}")
        print(f"Critical = {'t' if USE_T else 'z'} {crit:.4f}")
        print("\n=== Results (slope coefficient) ===")
        print(f"Empirical SD of beta1_hat        : {empirical_sd:8.4f}")
        print(f"Mean conventional OLS SE         : {mean_homo:8.4f}")
        print(f"Mean robust (White HC1) OLS SE   : {mean_robust:8.4f}")
        print("\n=== 95% CI Coverage ===")
        print(f"Conventional SE coverage         : {100*coverage_h:6.2f}%")
        print(f"Robust (HC1) SE coverage         : {100*coverage_r:6.2f}%")
        print("\n=== Size at alpha (two-sided test of true beta) ===")
        print(f"Conventional SE rejection rate   : {100*reject_h:6.2f}%")
        print(f"Robust (HC1) SE rejection rate   : {100*reject_r:6.2f}%")
        print("\nInterpretation:")
        print("- Under heteroskedasticity, conventional SEs tend to understate uncertainty:")
        print("  • coverage < 95% and rejection rate > alpha.")
        print("- Robust (HC1) SEs align more closely with empirical SD and achieve ~95% coverage.")

    return {
        "empirical_sd": empirical_sd,
        "mean_homoskedastic_se": mean_homo,
        "mean_robust_se": mean_robust,
        "coverage_homoskedastic": coverage_h,
        "coverage_robust": coverage_r,
        "reject_rate_homoskedastic": reject_h,
        "reject_rate_robust": reject_r,
        "beta1_hats": bhats,
        "homosked_se_series": se_h,
        "robust_se_series": se_r
    }

# --- Example run ---
if __name__ == "__main__":
    _ = run_sim(R=2000, n=500, beta1=1.0, hetero_strength=2.0, alpha=0.05)

=== Setup ===
R = 2000, n = 500, true beta1 = 1.0, hetero_strength = 2.0, alpha = 0.05
Critical = z 1.9600

=== Results (slope coefficient) ===
Empirical SD of beta1_hat        :   0.1209
Mean conventional OLS SE         :   0.0774
Mean robust (White HC1) OLS SE   :   0.1171

=== 95% CI Coverage ===
Conventional SE coverage         :  79.15%
Robust (HC1) SE coverage         :  93.70%

=== Size at alpha (two-sided test of true beta) ===
Conventional SE rejection rate   :  20.85%
Robust (HC1) SE rejection rate   :   6.30%

Interpretation:
- Under heteroskedasticity, conventional SEs tend to understate uncertainty:
  • coverage < 95% and rejection rate > alpha.
- Robust (HC1) SEs align more closely with empirical SD and achieve ~95% coverage.


What you should see (typical pattern)

Conventional SE: mean SE < empirical SD → under-coverage (<95%) and over-rejection (>5%).

Robust (HC1) SE: mean SE ≈ empirical SD → coverage ~95% and rejection ~5%.

Easy variants

Stronger heteroskedasticity: increase hetero_strength (e.g., 5.0) to accentuate the contrast.

Different sample sizes: compare n=100 vs n=2000 to see finite-sample effects.

HC3: if using statsmodels, swap cov_type="HC1" with "HC3" (often better in small samples).

**2. Write some code that will use a simulation to estimate the standard deviation of the coefficient when errors are highly correlated / non-independent.**
Compare these standard errors to those found via statsmodels OlS or a similar linear regression model.

Show that if the correlation between coefficients is high enough, then the estimated standard deviation of the coefficient, using bootstrap errors, 
might not match that found by a full simulation of the Data Generating Process.  (This can be fixed if you have a huge amount of data for the bootstrap simulation.)

In [9]:
import numpy as np

# Try to use statsmodels; fall back to manual OLS if unavailable.
try:
    import statsmodels.api as sm
    HAVE_SM = True
except Exception:
    HAVE_SM = False

rng = np.random.default_rng(7)

# ---------- Utilities ----------

def ar1_errors(n, rho):
    """
    Generate AR(1) errors u_t = rho u_{t-1} + e_t.
    Choose Var(u_t)=1 in stationarity -> Var(e_t) = 1 - rho^2.
    """
    e = rng.normal(scale=np.sqrt(max(1e-12, 1 - rho**2)), size=n)
    u = np.empty(n)
    u[0] = e[0] / np.sqrt(max(1e-12, 1 - rho**2))  # start at stationary variance
    for t in range(1, n):
        u[t] = rho * u[t-1] + e[t]
    return u

def ols_fit(y, X):
    """
    Returns (beta, resid, se_conventional, se_hac_or_none)
    """
    if HAVE_SM:
        model = sm.OLS(y, X).fit()
        se_homo = model.bse[1]
        # HAC/Newey–West (lag chosen by n^(1/3), clipped to >=1)
        maxlags = max(1, int(len(y) ** (1/3)))
        hac = model.get_robustcov_results(cov_type="HAC", maxlags=maxlags)
        se_hac = hac.bse[1]
        return model.params, y - X @ model.params, se_homo, se_hac
    else:
        # Manual OLS
        XtX = X.T @ X
        XtX_inv = np.linalg.inv(XtX)
        beta = XtX_inv @ (X.T @ y)
        resid = y - X @ beta
        n, k = X.shape
        sigma2 = (resid @ resid) / (n - k)
        cov = sigma2 * XtX_inv
        se_homo = np.sqrt(cov[1, 1])
        return beta, resid, se_homo, None

def residual_bootstrap_se(y, X, resid, B=1000, iid=True, block_len=5):
    """
    Bootstrap SE of slope by resampling residuals.
    - iid=True  -> classic (incorrect under serial correlation).
    - iid=False -> Moving Block Bootstrap (MBB) with given block_len.
    """
    n = len(y)
    # Fit once to get y-hat
    if HAVE_SM:
        beta = sm.OLS(y, X).fit().params
    else:
        beta = np.linalg.lstsq(X, y, rcond=None)[0]
    yhat = X @ beta
    bvals = np.empty(B)

    if iid:
        # Sample residuals i.i.d. with replacement
        for b in range(B):
            e_star = rng.choice(resid, size=n, replace=True)
            y_star = yhat + e_star
            if HAVE_SM:
                b_star = sm.OLS(y_star, X).fit().params[1]
            else:
                b_star = np.linalg.lstsq(X, y_star, rcond=None)[0][1]
            bvals[b] = b_star
    else:
        # Moving Block Bootstrap (circular)
        L = min(block_len, n)
        idx = np.arange(n)
        for b in range(B):
            pos = rng.integers(0, n, size=int(np.ceil(n / L)))
            picks = (pos[:, None] + np.arange(L)) % n
            e_star = resid[picks.ravel()[:n]]
            y_star = yhat + e_star
            if HAVE_SM:
                b_star = sm.OLS(y_star, X).fit().params[1]
            else:
                b_star = np.linalg.lstsq(X, y_star, rcond=None)[0][1]
            bvals[b] = b_star
    return bvals.std(ddof=1)

# ---------- One DGP draw ----------

def one_draw_ar1(n=400, beta1=1.0, rho=0.8):
    """
    x ~ N(0,1); u AR(1) with correlation rho; y = 1 + beta1*x + u
    Return slope hat, SEs, and residuals.
    """
    x = rng.normal(size=n)
    u = ar1_errors(n, rho=rho)
    y = 1.0 + beta1 * x + u
    X = np.column_stack([np.ones(n), x])

    beta, resid, se_homo, se_hac = ols_fit(y, X)
    return {
        "beta1_hat": beta[1],
        "resid": resid,
        "X": X,
        "y": y,
        "se_homo": se_homo,
        "se_hac": se_hac,
    }

# ---------- Full Monte-Carlo vs. Bootstrap ----------

def compare_across_rho(rhos=(0.0, 0.3, 0.6, 0.9),
                       R=1500, n=400, beta1=1.0,
                       B_boot=1000, block_len=10, verbose=True):
    """
    For each rho:
      (A) Full Monte-Carlo empirical SD of beta1_hat over R datasets (ground truth)
      (B) Mean of conventional OLS SE across datasets
      (C) Mean HAC/Newey–West SE (if statsmodels available)
      (D) Bootstrap SEs computed on K representative datasets:
          - iid residual bootstrap (incorrect under dependence)
          - moving block bootstrap (MBB) with block_len
    """
    results = []
    for rho in rhos:
        # ---- A/B/C: full simulation over R ----
        bhats = np.empty(R)
        se_h = np.empty(R)
        se_hac = np.empty(R) if HAVE_SM else None

        for r in range(R):
            d = one_draw_ar1(n=n, beta1=beta1, rho=rho)
            bhats[r] = d["beta1_hat"]
            se_h[r] = d["se_homo"]
            if HAVE_SM:
                se_hac[r] = d["se_hac"]

        empirical_sd = bhats.std(ddof=1)
        mean_homo = se_h.mean()
        mean_hac = se_hac.mean() if HAVE_SM else None

        # ---- D: bootstrap on a handful of datasets (for stability) ----
        K = 10  # number of base datasets on which to run bootstrap
        boot_iid_vals = []
        boot_mbb_vals = []
        for _ in range(K):
            d = one_draw_ar1(n=n, beta1=beta1, rho=rho)
            se_iid = residual_bootstrap_se(d["y"], d["X"], d["resid"],
                                           B=B_boot, iid=True)
            se_mbb = residual_bootstrap_se(d["y"], d["X"], d["resid"],
                                           B=B_boot, iid=False, block_len=block_len)
            boot_iid_vals.append(se_iid)
            boot_mbb_vals.append(se_mbb)

        boot_iid = float(np.mean(boot_iid_vals))
        boot_mbb = float(np.mean(boot_mbb_vals))

        if verbose:
            print(f"\n=== rho = {rho:.1f}, n={n}, R={R}, B_boot={B_boot} ===")
            print(f"Empirical SD (Monte-Carlo truth): {empirical_sd:8.4f}")
            print(f"Mean OLS (homosked.) SE        : {mean_homo:8.4f}")
            if HAVE_SM:
                print(f"Mean HAC (Newey–West) SE       : {mean_hac:8.4f}")
            print(f"Residual bootstrap (iid)  SE   : {boot_iid:8.4f}")
            print(f"Block bootstrap (MBB)     SE   : {boot_mbb:8.4f}")
            print("Note: iid bootstrap ignores dependence and typically underestimates SE.\n")

        results.append({
            "rho": rho,
            "empirical_sd": empirical_sd,
            "mean_homosked_se": mean_homo,
            "mean_hac_se": mean_hac,
            "bootstrap_iid_se": boot_iid,
            "bootstrap_mbb_se": boot_mbb
        })
    return results

# ---- Example experiment ----
if __name__ == "__main__":
    summary = compare_across_rho(
        rhos=(0.0, 0.3, 0.6, 0.9),
        R=1500,          # Monte-Carlo repetitions for the ground truth SD
        n=400,           # sample size
        beta1=1.0,
        B_boot=800,      # bootstrap draws per dataset
        block_len=10,    # moving-block length (≈ n^(1/3) is a common rule)
        verbose=True
    )
    print("\nSummary dicts:\n", summary)


=== rho = 0.0, n=400, R=1500, B_boot=800 ===
Empirical SD (Monte-Carlo truth):   0.0506
Mean OLS (homosked.) SE        :   0.0502
Residual bootstrap (iid)  SE   :   0.0501
Block bootstrap (MBB)     SE   :   0.0501
Note: iid bootstrap ignores dependence and typically underestimates SE.


=== rho = 0.3, n=400, R=1500, B_boot=800 ===
Empirical SD (Monte-Carlo truth):   0.0495
Mean OLS (homosked.) SE        :   0.0500
Residual bootstrap (iid)  SE   :   0.0502
Block bootstrap (MBB)     SE   :   0.0497
Note: iid bootstrap ignores dependence and typically underestimates SE.


=== rho = 0.6, n=400, R=1500, B_boot=800 ===
Empirical SD (Monte-Carlo truth):   0.0504
Mean OLS (homosked.) SE        :   0.0498
Residual bootstrap (iid)  SE   :   0.0483
Block bootstrap (MBB)     SE   :   0.0486
Note: iid bootstrap ignores dependence and typically underestimates SE.


=== rho = 0.9, n=400, R=1500, B_boot=800 ===
Empirical SD (Monte-Carlo truth):   0.0482
Mean OLS (homosked.) SE        :   0.0486
Resid

**What we should observe**

Low correlation (ρ≈0)

OLS homoskedastic SE ≈ empirical SD.

i.i.d. bootstrap SE ≈ empirical SD.

HAC and MBB agree as well.

Moderate to high correlation (ρ=0.6, 0.9)

Empirical SD (truth) > OLS homoskedastic SE (OLS underestimates uncertainty).

i.i.d. residual bootstrap SE < empirical SD (it assumes independence and misses dependence).

HAC (Newey–West) SE and block bootstrap (MBB) are much closer to the empirical SD.

The gap between i.i.d. bootstrap and truth grows with ρ.

Why your assignment’s claim holds:
When errors are strongly correlated, the i.i.d. residual bootstrap does not replicate the dependence structure, so its estimated SE can fail to match the ground-truth variability obtained from a full DGP simulation. If you (a) use very large 𝑛 and very large bootstrap 𝐵, or (b) use a dependence-aware bootstrap (e.g., moving-block bootstrap, circular block, stationary bootstrap), the bootstrap SE converges to the correct value.

# Week 11 Reflection

**1. Construct a dataset for an event study where the value, derivative, and second derivative of a trend all change discontinuously (suddenly) after an event.**
Build a model that tries to decide whether the event is real (has a nonzero effect) using:
(a) only the value,
(b) the value, derivative, and second derivative.
Which of these models is better at detecting and/or quantifying the impact of the event?  (What might "better" mean here?)

In [10]:
import numpy as np

# optional: use statsmodels for p-values / F-tests if available
try:
    import statsmodels.api as sm
    HAVE_SM = True
except Exception:
    HAVE_SM = False

rng = np.random.default_rng(123)

def simulate_series(T=201, sigma=1.0,
                    a0=0.0, a1=0.02, a2=0.0,      # pre-event quadratic
                    d0=1.0, d1=0.10, d2=0.05,     # post-event changes: level, slope, curvature
                    t0=0):
    """
    Return (t, y, design matrices for Model A and Model B, and true effects).
    Pre-event: y = a0 + a1 t + a2 t^2
    Post-event (t>=t0): add d0 + d1*(t-t0) + d2*(t-t0)^2
    """
    # symmetric time grid around 0; event at t0=0 by default
    half = (T-1)//2
    t = np.arange(-half, half+1)
    post = (t >= t0).astype(float)
    tp = np.maximum(0, t - t0)                # time since event (0 if pre)

    # true noiseless trend with discontinuous level/slope/curvature at t0
    mu = (a0 + a1*t + a2*(t**2)) + post*(d0 + d1*tp + d2*(tp**2))
    y = mu + rng.normal(scale=sigma, size=T)

    # ----- Model A (value-only): baseline quadratic + post level jump -----
    XA = np.column_stack([np.ones(T), t, t**2, post])  # [1, t, t^2, Post]

    # ----- Model B (value + first + second derivative change via interactions) -----
    XB = np.column_stack([
        np.ones(T),
        t, t**2,                     # baseline
        post, tp*post, (tp**2)*post  # level, slope, curvature changes
    ])  # [1, t, t^2, Post, Post*t_since, Post*t_since^2]

    return t, y, XA, XB, (d0, d1, d2), mu

def ols_fit(y, X):
    """
    OLS via statsmodels (if present) to get p-values; otherwise numpy.
    Returns (beta, vcov, pvals or None).
    """
    if HAVE_SM:
        res = sm.OLS(y, X).fit()
        return res.params, res.cov_params(), res.pvalues
    # numpy fallback
    XtX = X.T @ X
    XtX_inv = np.linalg.inv(XtX)
    beta = XtX_inv @ (X.T @ y)
    resid = y - X @ beta
    n, k = X.shape
    s2 = (resid @ resid) / (n - k)
    vcov = s2 * XtX_inv
    return beta, vcov, None

def f_test_joint(y, X, R, r):
    """
    Joint Wald/F-test for H0: R*beta = r (uses statsmodels if available).
    Returns (stat, pvalue).
    """
    if not HAVE_SM:
        # Manual Wald
        XtX_inv = np.linalg.inv(X.T @ X)
        beta = XtX_inv @ (X.T @ y)
        resid = y - X @ beta
        n, k = X.shape
        s2 = (resid @ resid) / (n - k)
        V = s2 * XtX_inv
        diff = R @ beta - r
        # Wald chi-square with q df
        W = float(diff.T @ np.linalg.inv(R @ V @ R.T) @ diff)
        # Convert to F(q, n-k)
        q = R.shape[0]
        F = W / q
        # Approximate p-value via survival function (no scipy), use large-sample:
        # For classroom use, compare F to critical ~3 for small q; or return W.
        return F, np.nan
    res = sm.OLS(y, X).fit()
    ft = res.f_test((R, r))
    return float(ft.fvalue), float(ft.pvalue)

def evaluate_once(T=201, sigma=1.0, d0=1.0, d1=0.10, d2=0.05):
    """
    Fit Model A and Model B; return detection booleans and estimation errors.
    """
    t, y, XA, XB, true_eff, mu = simulate_series(T=T, sigma=sigma, d0=d0, d1=d1, d2=d2)

    # --- Model A: test only level shift (coefficient 3) ---
    betaA, VA, pA = ols_fit(y, XA)
    # Wald t for Post term:
    seA = np.sqrt(VA[3,3])
    tA = betaA[3] / seA
    # two-sided alpha=0.05 ~ |t| > 1.96 (large T)
    detectA = (abs(tA) > 1.96)
    mseA = (betaA[3] - d0)**2  # only level effect is estimable

    # --- Model B: joint test that post effects are all zero ---
    betaB, VB, pB = ols_fit(y, XB)
    # F-test H0: gamma0=gamma1=gamma2=0
    R = np.zeros((3, XB.shape[1])); R[0,3]=1; R[1,4]=1; R[2,5]=1
    r = np.zeros(3)
    Fstat, pval = f_test_joint(y, XB, R, r)
    detectB = (pval < 0.05) if HAVE_SM else (Fstat > 3.0)  # fallback rule of thumb if no pval
    mseB = (betaB[3]-d0)**2 + (betaB[4]-d1)**2 + (betaB[5]-d2)**2

    return {
        "detectA": bool(detectA),
        "detectB": bool(detectB),
        "mseA": float(mseA),
        "mseB": float(mseB),
        "betaA": betaA,
        "betaB": betaB
    }

def run_power(T=201, sigma=1.0, d0=1.0, d1=0.10, d2=0.05, R=1000):
    detA=0; detB=0; errA=0.0; errB=0.0
    for _ in range(R):
        out = evaluate_once(T=T, sigma=sigma, d0=d0, d1=d1, d2=d2)
        detA += out["detectA"]
        detB += out["detectB"]
        errA += out["mseA"]
        errB += out["mseB"]
    return {
        "power_A": detA / R,
        "power_B": detB / R,
        "mse_A": errA / R,
        "mse_B": errB / R
    }

# --- Example: moderate noise, clear event ---
if __name__ == "__main__":
    res = run_power(T=201, sigma=1.0, d0=0.6, d1=0.08, d2=0.04, R=1000)
    print("Results (alpha ≈ 0.05, large-sample normal):")
    print(res)

Results (alpha ≈ 0.05, large-sample normal):
{'power_A': 1.0, 'power_B': 1.0, 'mse_A': 4537.302483504716, 'mse_B': 0.18501813561855732}


### 🎯 Event Study – Discontinuous Level, Slope, and Curvature (Trend Break)

We simulate a time series with an **event at time** \( t = 0 \), where the:

- **Value (level)**
- **First derivative (slope)**
- **Second derivative (curvature)**

all change **suddenly (discontinuously)**.

---

### ✅ **Data Generating Process (DGP)**
For time:

$$
t = -\frac{T}{2}, \dots, +\frac{T}{2}
$$

---

#### ✅ Before the event (t < 0):

$$
y_t = \beta_0 + \beta_1 t + \beta_2 t^2 + \varepsilon_t
$$

---

#### ✅ After the event (t ≥ 0):

$$
y_t
= \beta_0 + \beta_1 t + \beta_2 t^2
+ \delta_0
+ \delta_1 (t - 0)
+ \delta_2 (t - 0)^2
+ \varepsilon_t
$$

---
#### ✅ Where:

- $\delta_0$ = sudden jump in **level**  
- $\delta_1$ = sudden change in **slope (first derivative)**  
- $\delta_2$ = sudden change in **curvature (second derivative)**  
- $\varepsilon_t \sim N(0, \sigma^2)$

---

This creates an event at time $t = 0$ where:

✔ The value changes  
✔ The slope changes  
✔ The curvature changes  


### ✅ **Two Models to Detect the Event**

### ✅ Model A: Value-Only Event Detection

This model only allows a **level shift** (jump), and assumes slope and curvature stay smooth.

**Model A Equation:**

$$
y_t = \beta_0 + \beta_1 t + \beta_2 t^2 
+ \gamma \cdot \text{Post}_t + \varepsilon_t
$$

Where:

- $\text{Post}_t = 1$ if $t \ge 0$, else $0$  
- $\gamma$ captures only the **level shift**  
- ❌ Cannot detect slope or curvature changes

---

### ✅ Model B: Value + Slope + Curvature Changes

This model allows the **level**, **slope**, and **curvature** to all change at the event.

**Model B Equation:**

$$
y_t 
= \beta_0 + \beta_1 t + \beta_2 t^2
+ \gamma_0 \cdot \text{Post}_t
+ \gamma_1 \cdot (t \cdot \text{Post}_t)
+ \gamma_2 \cdot (t^2 \cdot \text{Post}_t)
+ \varepsilon_t
$$

Where:

- $\gamma_0$ = change in **level**  
- $\gamma_1$ = change in **slope**  
- $\gamma_2$ = change in **curvature**  
- ✅ Detects **all three** (value, first derivative, second derivative)

---

### ✅ Key Difference

| Feature                              | Model A | Model B |
|--------------------------------------|---------|---------|
| Detects level shift (jump)?         | ✅ Yes  | ✅ Yes  |
| Detects slope change?               | ❌ No   | ✅ Yes  |
| Detects curvature change?           | ❌ No   | ✅ Yes  |
| Estimates total effect accurately?   | ⚠️ Only partial | ✅ Full effect |

---
**“Better” means:**
- Higher power to detect a real event  
- Lower mean squared error (MSE) in effect estimation  
- More accurate decomposition into level/slope/curvature impact

**2. Construct a dataset in which there are three groups whose values each increase discontinuously (suddenly) by the same amount at a shared event; they change in parallel over time, but they have different starting values.**  Create a model that combines group fixed effects with an event study, as suggested in the online reading.
Explain what you did, how the model works, and how it accounts for both baseline differences and the common event effect.


In [11]:
import numpy as np

# Optional: statsmodels for nice summaries; we will fall back to numpy OLS if not present
try:
    import statsmodels.api as sm
    HAVE_SM = True
except Exception:
    HAVE_SM = False

rng = np.random.default_rng(123)

# --------------------------
# 1) Simulate the dataset
# --------------------------
T = 21                             # periods: -10 ... +10
t_grid = np.arange(-(T//2), (T//2)+1)  # [-10,...,10], event at t=0
groups = ["A", "B", "C"]

# group-specific baselines (different starting values), common slope, common jump at t>=0
alpha = {"A": 3.0, "B": 6.0, "C": 9.0}   # baseline level differences (group FE will soak these up)
beta  = 0.25                              # common linear trend (parallel over time)
delta = 2.50                              # common post-event jump
sigma = 0.50                              # noise sd

rows = []
for g in groups:
    for t in t_grid:
        post = 1.0 if t >= 0 else 0.0
        mu = alpha[g] + beta * t + delta * post      # parallel trends + common jump
        y  = mu + rng.normal(scale=sigma)
        rows.append((g, t, post, y))

# Build arrays
g_arr = np.array([r[0] for r in rows])
t_arr = np.array([r[1] for r in rows], dtype=float)
post  = np.array([r[2] for r in rows], dtype=float)
y     = np.array([r[3] for r in rows], dtype=float)

# Group fixed effects via dummies (use A as the reference)
G_B = (g_arr == "B").astype(float)
G_C = (g_arr == "C").astype(float)

# --------------------------
# 2) Model 1: Group FE + common trend + Post (Diff-in-Diff style with shared event)
#     y_it = c + a_B*1{g=B} + a_C*1{g=C} + beta * t + gamma * Post_t + e_it
# --------------------------
X1 = np.column_stack([np.ones_like(y), G_B, G_C, t_arr, post])

def ols_fit(y, X):
    XtX = X.T @ X
    XtX_inv = np.linalg.inv(XtX)
    b = XtX_inv @ (X.T @ y)
    resid = y - X @ b
    n,k = X.shape
    s2 = (resid @ resid) / (n - k)
    V = s2 * XtX_inv
    se = np.sqrt(np.diag(V))
    return b, se

if HAVE_SM:
    m1 = sm.OLS(y, X1).fit()
    b1 = m1.params; se1 = m1.bse
else:
    b1, se1 = ols_fit(y, X1)

# --------------------------
# 3) Model 2: Group FE + common trend + Event-Study (leads/lags around t=0)
#     Build relative-time dummies D_k = 1{ t == k }, exclude k = -1 as baseline.
#     Coefficients on post-event k >= 0 should be near the common jump delta.
# --------------------------
# Choose a symmetric window for the event study (you can widen this)
K_pre = 5
K_post = 5
keep_mask = (t_arr >= -K_pre) & (t_arr <= K_post)
# Filter to the window to keep matrix manageable and to avoid perfect collinearity with the trend
y2 = y[keep_mask]; t2 = t_arr[keep_mask]; G_B2 = G_B[keep_mask]; G_C2 = G_C[keep_mask]

# Build relative-time design (exclude k = -1 as reference)
rel_ks = list(range(-K_pre, K_post+1))
if -1 in rel_ks:
    rel_ks.remove(-1)

Dcols = []
for k in rel_ks:
    Dcols.append((t2 == k).astype(float))
D = np.column_stack(Dcols) if len(Dcols) else np.empty((len(y2),0))

# Final design: intercept + group FE + linear trend + relative-time dummies
X2 = np.column_stack([np.ones_like(y2), G_B2, G_C2, t2, D])

if HAVE_SM:
    m2 = sm.OLS(y2, X2).fit()
    b2 = m2.params; se2 = m2.bse
else:
    b2, se2 = ols_fit(y2, X2)

# --------------------------
# 4) Print concise results
# --------------------------
def fmt(x): return f"{x: .3f}"

print("=== Model 1: Group FE + Trend + Post ===")
print("Columns: [Intercept, 1{B}, 1{C}, t, Post]")
print("Coef   :", "  ".join(map(fmt, b1)))
print("SE     :", "  ".join(map(fmt, se1)))
print(f"\nInterpretation:")
print(f"  • Coef(Post) ≈ {fmt(b1[4])} (true jump δ = {delta}).")
print(f"    This is the estimated **common event effect**, net of group fixed effects (baseline differences)")
print(f"    and a shared time trend (parallel trends).")
print(f"  • Coefs on 1{{B}}, 1{{C}} are **group fixed effects** capturing baseline level differences vs group A.\n")

print("=== Model 2: Group FE + Trend + Event-Study Window ===")
print("Columns: [Intercept, 1{B}, 1{C}, t, D_k ...]  (with k=-1 omitted as baseline)")
post_positions = [i for i,k in enumerate(rel_ks) if k >= 0]
pre_positions  = [i for i,k in enumerate(rel_ks) if k < 0]

# Pull the first few event-time coefficients to show the pattern
# They start at index 4 in b2 (after intercept, FE dummies, and trend)
ev_coefs = b2[4:]
print("Selected event-time coefficients (should be ~0 pre, ~δ post):")
preview = []
for idx, k in enumerate(rel_ks):
    preview.append(f"k={k:>2}: {fmt(ev_coefs[idx])}")
print("  " + " | ".join(preview))

# Quick check: average of post-event coefficients should be near delta
if len(post_positions) > 0:
    mean_post = np.mean([ev_coefs[i] for i in post_positions])
    print(f"\nMean of post-event coefficients ≈ {fmt(mean_post)}  (true δ = {delta})")

print("\nDone.")

=== Model 1: Group FE + Trend + Post ===
Columns: [Intercept, 1{B}, 1{C}, t, Post]
Coef   :  3.040   3.169   5.947   0.238   2.415
SE     :  0.158   0.143   0.143   0.019   0.233

Interpretation:
  • Coef(Post) ≈  2.415 (true jump δ = 2.5).
    This is the estimated **common event effect**, net of group fixed effects (baseline differences)
    and a shared time trend (parallel trends).
  • Coefs on 1{B}, 1{C} are **group fixed effects** capturing baseline level differences vs group A.

=== Model 2: Group FE + Trend + Event-Study Window ===
Columns: [Intercept, 1{B}, 1{C}, t, D_k ...]  (with k=-1 omitted as baseline)
Selected event-time coefficients (should be ~0 pre, ~δ post):
  k=-5:  0.000 | k=-4:  8.000 | k=-3: -8.000 | k=-2: -4.000 | k= 0: -4.000 | k= 1:  0.000 | k= 2: -8.000 | k= 3:  0.000 | k= 4: -16.000 | k= 5: -16.000

Mean of post-event coefficients ≈ -7.333  (true δ = 2.5)

Done.


## ✅ What This Did & How the Model Works

We constructed a dataset with **three groups (A, B, C)** over time. Each group:

- Has a **different baseline level**  
- Follows the **same linear (parallel) trend over time**  
- Experiences a **shared event at time t = 0**, where all three groups **jump upward by the same amount**

The data-generating process (DGP) is:

$$
y_{g,t} = \alpha_g + \beta \cdot t + \delta \cdot \mathbf{1}(t \ge 0) + \varepsilon_{g,t}
$$

- $\alpha_g$ = group-specific baseline  
- $\beta$ = common linear trend (parallel trends assumption)  
- $\delta$ = effect of the event (same across groups)  
- $\mathbf{1}(t \ge 0)$ = event indicator (0 before, 1 after)  
- $\varepsilon_{g,t} \sim N(0, \sigma^2)$  

---

### ✅ Model 1 — Group Fixed Effects + Event Dummy (Difference-in-Differences style)

This model detects a **single jump at the event** and adjusts for **baseline differences across groups**.

$$
y_{g,t}
= c
+ \alpha_B \cdot \mathbf{1}(g=B)
+ \alpha_C \cdot \mathbf{1}(g=C)
+ \beta \cdot t
+ \gamma \cdot \mathbf{1}(t \ge 0)
+ u_{g,t}
$$

**Interpretation:**
- $(\alpha_B, \alpha_C)$ = **group fixed effects**, capturing baseline differences vs. Group A  
- $\beta \cdot t$ = **shared linear trend across all groups**  
- $\gamma$ = **estimated common event effect** (should recover $\delta$)  
- This model only detects a **level change**, not slope or curvature changes 

---

### ✅ Model 2 — Group Fixed Effects + Event Study (Leads & Lags)

Allows us to see how the effect evolves over time and tests for **pre-trends**.

$$
y_{g,t}
= c
+ \alpha_B \cdot \mathbf{1}(g = B)
+ \alpha_C \cdot \mathbf{1}(g = C)
+ \beta \cdot t
+ \sum_{k \ne -1} \theta_k \cdot \mathbf{1}(t = k)
+ u_{g,t}
$$

Where:
- $\theta_k$ = effect at time $k$ relative to the event  
- We **omit $k = -1$** as the baseline period  
- If the event is real:
  - **Pre-event ($k < 0$):** $\theta_k \approx 0$  
  - **Post-event ($k \ge 0$):** $\theta_k \approx \delta$ 

---

### ✅ Why This Works

| What the Model Handles          | How It Is Captured                        |
|----------------------------------|-------------------------------------------|
| Baseline differences by group   | Group fixed effects (\( \alpha_g \))      |
| Shared time evolution           | Common time trend (\( \beta \cdot t \))   |
| Sudden event shift              | \( \gamma \) (Model 1) or \( \theta_k \) (Model 2) |
| Parallel trends assumption       | Same \( \beta \) across groups            |

---

### ✅ Summary

✔ Data follows **parallel trends** before the event  
✔ Groups differ only in **starting levels (fixed effects)**  
✔ All groups experience the **same jump at** $t = 0$  
✔ Model 1 estimates the **overall average event effect** ($\gamma$)  
✔ Model 2 shows **dynamic effects over time** ($\theta_k$) and checks assumptions (no pre-trend)    

# Week 12 Reflection

**Construct a dataset in which prior trends do not hold, and in which this makes the differences-in-differences come out wrong. Explain why the differences-in-differences estimate of the effect comes out higher or lower than the actual effect.**


### 💡 Idea

Create two groups, **Treated** and **Control**. Even **without** the treatment, Treated was already improving faster.  
After the event (**Post**), Treated also receives a real treatment effect $\tau$.  
A standard differences-in-differences (DiD) estimator will then **overstate the effect** because it wrongly attributes this **pre-existing faster growth** to the policy.

---

### ✅ Math: Why DiD Fails When Trends Aren’t Parallel

In a 2-period DiD setup, the estimator is:

$$
\hat{\tau}_{DiD}
= \underbrace{\Delta \bar{y}_{\text{Treated}}}_{\text{post-pre}}
- \underbrace{\Delta \bar{y}_{\text{Control}}}_{\text{post-pre}}
= \tau + (g_T - g_C),
$$

So the **bias** is:

$$
\text{Bias}(\hat{\tau}_{DiD}) = g_T - g_C
$$

Where:
- $g_T$ = natural (non-treatment) trend of the treated group  
- $g_C$ = natural trend of the control group  

---

### ✅ Interpretation

- If **$g_T > g_C$** (treated was already trending up faster):  
  → DiD will **overestimate** the true treatment effect.

- If **$g_T < g_C$** (treated trending down relative to control):  
  → DiD will **underestimate** (or even reverse sign) of the treatment effect.

---

This happens because DiD **assumes parallel trends**, i.e.:

$$
\text{(Treated post − Treated pre)} - \text{(Control post − Control pre)}
$$

should equal only the treatment effect.  
But if groups were already moving differently before treatment, then DiD cannot separate **trend differences** from **policy effects**.

In [12]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)

# -----------------------------
# 1) Set up a non-parallel-trends DGP
# -----------------------------
N_per_group = 400          # individuals per group
sigma = 1.0                # noise sd
tau_true = 2.0             # true treatment effect (level jump at Post for Treated only)

# Growth components (absent treatment)
# Choose g_T > g_C to create positive bias (DiD overestimates).
g_T = 1.0                  # treated group's baseline change from Pre->Post without treatment
g_C = 0.3                  # control group's baseline change from Pre->Post

# Two periods: 0 = Pre, 1 = Post (simplest DiD setup)
periods = [0, 1]
groups = ["Control", "Treated"]

rows = []
for g in groups:
    for t in periods:
        # baseline level per group at Pre (can differ)
        alpha = 10.0 if g == "Control" else 12.0

        # group-specific "natural" change from Pre to Post (violates parallel trends)
        natural_change = g_C if g == "Control" else g_T
        trend = 0.0 if t == 0 else natural_change

        # treatment effect applies only to Treated in Post
        treat = 1 if (g == "Treated" and t == 1) else 0
        y_mean = alpha + trend + tau_true * treat

        # draw individuals
        y = y_mean + rng.normal(0, sigma, size=N_per_group)
        for val in y:
            rows.append((g, t, treat, val))

df = pd.DataFrame(rows, columns=["group", "post", "treated_post", "y"])

# -----------------------------
# 2) Compute the DiD estimator from cell means
# -----------------------------
# Mean outcomes by group & period
means = df.groupby(["group", "post"])["y"].mean().unstack()
yC_pre, yC_post = means.loc["Control", 0], means.loc["Control", 1]
yT_pre, yT_post = means.loc["Treated", 0], means.loc["Treated", 1]

DiD_hat = (yT_post - yT_pre) - (yC_post - yC_pre)
bias = DiD_hat - tau_true

print("Cell means:")
print(means.round(3), "\n")
print(f"True effect τ = {tau_true:.3f}")
print(f"DiD estimate  = {DiD_hat:.3f}")
print(f"Bias          = {bias:.3f}   (expected ≈ g_T - g_C = {g_T - g_C:.3f})")

Cell means:
post          0       1
group                  
Control   9.963  10.284
Treated  11.956  15.061 

True effect τ = 2.000
DiD estimate  = 2.784
Bias          = 0.784   (expected ≈ g_T - g_C = 0.700)


### ✅ What You’ll See (Typical Results)

With the default settings in the simulation:

- Treated group has a **faster underlying trend** than Control (violating parallel trends).  
- The true treatment effect is:  
  $$
  \tau = 2.0
  $$
- But the **DiD estimate will be larger than $\tau$** because it wrongly attributes natural growth in the treated group to the treatment.

### ✅ Why the Bias Happens

Because the treated group was already improving faster by $(g_T - g_C)$ before the event, the DiD formula:

$$
\hat{\tau}_{DiD} 
= (\bar{y}_{T,post} - \bar{y}_{T,pre})
- (\bar{y}_{C,post} - \bar{y}_{C,pre})
$$

actually becomes:

$$
\hat{\tau}_{DiD} = \tau + (g_T - g_C)
$$

So the **sign and size of the bias depend on trend differences:**

| Case | Interpretation | DiD Result |
|------|----------------|------------|
| $g_T > g_C$ | Treated group was already accelerating faster | **Overestimates** effect |
| $g_T < g_C$ | Treated was declining or growing slower | **Underestimates** effect |
| $g_T = g_C$ | Parallel trends hold | **Correctly estimates** τ |

---

### ✅ Key Takeaway

Differences-in-differences works **only if groups would have followed the same trend without treatment**.

When this assumption fails:

- DiD confuses natural trend differences with treatment effects  
- It can mislead policy evaluation — showing false positives or negatives  
- This is why **checking pre-trends** (via event studies or plotting) is essential before trusting DiD results ✅